<a href="https://colab.research.google.com/github/SnrPep/Mos/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
text = "i like pizza and i like pasta but i hate broccoli"
words = text.split()
vocab = sorted(set(words))
stoi = {w: i for i, w in enumerate(vocab)}
itos = {i: w for w, i in stoi.items()}
vocab_size = len(vocab)

In [4]:
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __abs__(self):
        out = Value(abs(self.data), (self,), 'abs')
        def _backward():
            self.grad += (1 if self.data >= 0 else -1) * out.grad
        out._backward = _backward
        return out

    def tanh(self):
        t = (math.exp(2 * self.data) - 1) / (math.exp(2 * self.data) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t ** 2) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)
        build(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(0.0)

    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        return [n(x) for n in self.neurons]

class NeuralNet:
    def __init__(self, sizes):
        self.layers = []
        for i in range(len(sizes) - 1):
            self.layers.append(Layer(sizes[i], sizes[i + 1]))

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for neuron in layer.neurons for p in neuron.w + [neuron.b]]


In [5]:
def one_hot(idx, size):
    return [Value(1.0 if i == idx else 0.0) for i in range(size)]

context_len = 2
X, y = [], []
for i in range(len(words) - context_len):
    ctx = words[i:i+context_len]
    tgt = words[i+context_len]
    x = []
    for w in ctx:
        x.extend(one_hot(stoi[w], vocab_size))
    X.append(x)
    y.append(stoi[tgt])

In [7]:
def cross_entropy(preds, target_idx):
    exps = [math.exp(p.data) for p in preds]
    total = sum(exps)
    probs = [e / total for e in exps]
    return Value(-math.log(probs[target_idx]))

In [8]:
net = NeuralNet([vocab_size * context_len, 32, 32, vocab_size])
lr = 0.05

for epoch in range(300):
    total_loss = 0
    for xi, yi in zip(X, y):
        out = net(xi)
        loss = cross_entropy(out, yi)

        for p in net.parameters():
            p.grad = 0.0
        loss.backward()
        for p in net.parameters():
            p.data -= lr * p.grad

        total_loss += loss.data
    if epoch % 50 == 0:
        print(f"Epoch {epoch} | Loss: {total_loss/len(X):.4f}")


Epoch 0 | Loss: 2.3942
Epoch 50 | Loss: 2.3942
Epoch 100 | Loss: 2.3942
Epoch 150 | Loss: 2.3942
Epoch 200 | Loss: 2.3942
Epoch 250 | Loss: 2.3942


In [9]:
def generate(start_words, steps=5):
    context = start_words[-context_len:]
    result = context.copy()
    for _ in range(steps):
        x = []
        for w in context:
            x.extend(one_hot(stoi[w], vocab_size))
        out = net(x)
        probs = [math.exp(p.data) for p in out]
        total = sum(probs)
        probs = [p / total for p in probs]
        idx = random.choices(range(vocab_size), weights=probs)[0]
        word = itos[idx]
        result.append(word)
        context = result[-context_len:]
    return ' '.join(result)

print("\nСгенерированный текст:")
print(generate(["i", "like"], 10))


Сгенерированный текст:
i like like and like pasta but hate hate pasta pasta i
